# Project MedVision: Radiology AI Assistant Lab

Welcome back! In Notebook 01 you built a structured patient chart and crafted a handover note.
Today you will hand that context to our imaging AI engine and evaluate how well it assists the care team.

## Mission Brief

- Initialise the MedVision AI tooling inside Colab using the same self-contained setup pattern.
- Reuse the patient summary you assembled earlier to ground every imaging prompt.
- Experiment with prompts, personas, and images while logging clinical insights.
- Benchmark MedGemma against an experimental LLaVA engine and make a deployment recommendation.

### Why This Lab Matters: The Future of Diagnostic Collaboration

Radiology AI is already deployed in hospitals worldwide for:
- **Triage:** Flagging critical findings like pneumothorax for urgent reads
- **Quality assurance:** Catching missed findings on "normal" reports  
- **Workflow optimization:** Pre-populating draft reports for radiologist refinement

**Your generation will inherit these tools.** This lab teaches you to:
- Operate AI systems with clinical discipline
- Evaluate outputs with appropriate skepticism
- Communicate findings that blend AI insights with human judgment

**The Goal:** Not to replace radiologists, but to help them focus expertise where it matters most while maintaining safety standards you'd want for your own family.

### Learning Objectives

By the end of this lab, you will:
1. Understand how vision-language models "see" and "interpret" medical images
2. Craft effective prompts that ground AI analysis in clinical context
3. Recognize AI hallucinations and apply verification protocols
4. Evaluate competing AI models using clinical decision criteria
5. Integrate AI-assisted reads into safe clinical workflows

**Estimated time:** 90-120 minutes

### How to Use This Notebook
- Run cells from top to bottom; restart the runtime if you change dependencies.
- Challenge cells intentionally raise `NotImplementedError` until you complete them—this keeps the workflow honest.
- Solutions live in collapsed cells. Treat them as a safety net, not a shortcut.
- Keep the handover note from Notebook 01 close by; you will reference it throughout the lab.

## 0. Setup: Provision Your Colab Runtime

This cell clones (or updates) the MedVision repository, installs dependencies, and ensures the toolkit is importable.

In [ ]:
from __future__ import annotations

import subprocess
import sys
from pathlib import Path
from typing import Final

REPO_URL: Final[str] = "https://github.com/your-repo/python-for-medicine-bootcamp.git"

workspace = Path.cwd()

# Try to find the project root
# Case 1: Running from project root (workspace contains src/medvision_toolkit)
# Case 2: Running from notebooks/ subdirectory (parent contains src/medvision_toolkit)
# Case 3: Running in Colab (need to clone repository)

if (workspace / "src" / "medvision_toolkit").exists():
    repo_root = workspace
    print("✓ Detected Project MedVision source in the current workspace.")
elif (workspace.parent / "src" / "medvision_toolkit").exists():
    repo_root = workspace.parent
    print("✓ Detected Project MedVision source in parent directory.")
    print(f"  Project root: {repo_root}")
else:
    # Colab or git clone scenario
    repo_root = workspace / "python-for-medicine-bootcamp"
    if repo_root.exists():
        print("Existing checkout found. Pulling latest changes...")
        result = subprocess.run(
            ["git", "-C", str(repo_root), "pull", "--ff-only"],
            capture_output=True,
            text=True,
        )
        if result.returncode != 0:
            print(result.stdout)
            print(result.stderr, file=sys.stderr)
            raise RuntimeError("git pull failed; please resolve and rerun the cell.")
    else:
        print(f"Cloning repository from {REPO_URL} …")
        result = subprocess.run(
            ["git", "clone", "--depth", "1", REPO_URL, str(repo_root)],
            capture_output=True,
            text=True,
        )
        if result.returncode != 0:
            print(result.stdout)
            print(result.stderr, file=sys.stderr)
            raise RuntimeError("git clone failed; check the URL or your connection.")

# Install dependencies (only in Colab or if explicitly needed)
requirements_path = repo_root / "requirements.txt"
if not requirements_path.exists():
    raise FileNotFoundError(f"Could not locate requirements.txt at {requirements_path}")

# Check if we're in Colab (has google.colab module)
try:
    import google.colab
    in_colab = True
except ImportError:
    in_colab = False

if in_colab:
    print("Installing Python dependencies (this may take a minute)…")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", str(requirements_path)])
else:
    print("Running locally. Assuming dependencies are already installed.")
    print("If you encounter import errors, run: pip install -r requirements.txt")

# Add src directory to path
project_src = repo_root / "src"
if str(project_src) not in sys.path:
    sys.path.insert(0, str(project_src))
    print(f"✓ Added {project_src} to Python path")

print("✅ Toolkit ready. Continue to the next cell.")

### Quick Systems Check

Confirm the toolkit loads, summarise the patient profile, and preview tonight's handover so the AI work stays grounded in clinical reality.

In [ ]:
from medvision_toolkit import (
    draft_handover_for_night_team,
    load_sample_patient,
    summarize_patient,
)

patient_profile = load_sample_patient()
print(summarize_patient(patient_profile))

handover_note = draft_handover_for_night_team(patient_profile)
print("\nNight Handover Preview:\n")
print(handover_note)


## 0.5. Before We Start: Understanding Medical AI

Before you interact with the RadiologyAI engine, you need to understand what you're working with—and more importantly, what its limitations are.

### Vision-Language Models: The Technology Behind the Assistant

The RadiologyAI engine you're about to use is a **vision-language model (VLM)**—an AI system trained to:

1. **"See"** patterns in images (like infiltrates, device placement, cardiac silhouettes)
2. **"Understand"** your clinical questions in natural language  
3. **"Respond"** with structured text findings

**Key Insight:** Unlike rule-based systems or clinical decision tools, VLMs use **statistical pattern recognition**. They don't "know" medicine in the way a radiologist does—they've learned associations from analyzing millions of images and text pairs during training.

Think of it like this:
- A radiologist **understands** that hazy opacities in a certain distribution suggest pulmonary edema because they know cardiac physiology
- A VLM has **learned** that images with those visual patterns are often described using words like "pulmonary edema" in radiology reports

The VLM can be remarkably helpful, but it's pattern-matching, not reasoning.

### What is GGUF? Why Does It Matter in Colab?

You'll see references to "GGUF" throughout this lab. Here's what you need to know:

- **GGUF** = A quantized model format that trades some numerical precision for much smaller file sizes
- **Full MedGemma model:** ~10GB (won't fit in free Colab's memory)
- **GGUF quantized version:** ~4GB (perfect for teaching environments)
- **The trade-off:** Slightly less nuanced outputs, but 95% as capable for learning purposes

**Why this matters:** Quantization makes cutting-edge AI accessible for education and prototyping. In production radiology departments, you'd typically use the full-precision model on dedicated GPU servers.

### The Three Pillars of Safe AI-Assisted Radiology

⚠️ **CRITICAL SAFETY PRINCIPLES**

Every AI-generated finding in this lab is **preliminary** and requires:

1. **Verification:** A qualified radiologist must review the source images and confirm or revise AI-suggested findings
2. **Clinical Correlation:** AI outputs must be interpreted in the context of patient history, physical exam, labs, and prior imaging
3. **Documentation:** Any clinical decision informed by AI must document that AI was used as a decision support tool

**In Real Practice:** You would never act on an AI read alone. Think of the AI as a smart intern's draft report—useful for catching things you might have missed, but you always verify their work before signing off.

**Scope of Practice:** As medical students and junior doctors, you are learning to consume AI outputs critically. Interpreting imaging studies and making diagnostic conclusions remains the domain of qualified radiologists and supervising physicians.

### Common AI Failure Modes You'll Encounter

**1. Hallucinations:** The AI confidently reports findings that aren't actually present in the image

**2. Overconfidence:** Stating definitive diagnoses when findings are subtle or ambiguous  

**3. Measurement Errors:** Quantifying things (angles, volumes, dimensions) without actual measurement tools

**4. Context Blindness:** Missing clinically relevant details because the prompt didn't explicitly ask about them

**5. Rare Finding Struggles:** Performing poorly on uncommon pathologies not well-represented in training data

You'll practice detecting these failure modes throughout the lab.

### Reflection Question (Think Before Moving On)

**Scenario:** An overnight AI system flags a chest X-ray as "likely normal" and deprioritizes it in the reading queue. The next morning, a radiologist reviews it and identifies a small pneumothorax that the AI missed.

Who is responsible? What safeguards should be in place?

*There's no single right answer, but thinking through this now will make you a better steward of these tools.*

---

**Now that you understand what you're working with—and what could go wrong—let's begin the lab with clear eyes.**

## 1. Onboarding: Spin Up the Radiology Engine

We abstracted the heavy lifting into helper functions. Initialising the engine should feel like calling a standard protocol.

In [ ]:
from medvision_toolkit import (
    build_radiology_prompt,
    initialize_medgemma_engine,
    load_and_display_image,
    render_ai_report,
)

print("--- Initialising MedGemma GGUF Engine ---")
medgemma_engine = initialize_medgemma_engine(backend="gguf")

### What Happens During Initialization?

When you run the next cell, the toolkit will execute several steps behind the scenes:

**Step 1: Model Download** (~3-5 minutes on first run)
- The GGUF-quantized MedGemma model (~4GB) downloads from Hugging Face Hub
- Subsequent runs are instant—the model is cached in Colab's storage
- If you restart the runtime, the cache persists for this session

**Step 2: Model Loading** (~30 seconds)
- The model loads into Colab's RAM (~10GB total memory required)
- The inference engine compiles optimizations for your hardware
- A warmup inference runs to ensure everything works

**Step 3: Validation** (~5 seconds)
- The toolkit runs a quick test to confirm the model can generate text
- If successful, you'll see "Engine initialized" confirmation

**⏱️ Total Expected Time:**
- First run: 5-10 minutes (mostly download time)
- Subsequent runs: 30-45 seconds (loading only)

**💾 Colab Resource Check:**

Free Colab provides:
- **RAM:** 12-13GB (we need ~10GB for MedGemma)
- **Disk:** 100GB+ (plenty for our 8GB model)
- **Session duration:** 12 hours max, 90 minutes idle timeout

**🚨 Troubleshooting Common Issues:**

| Problem | Likely Cause | Solution |
|---------|-------------|----------|
| "Out of memory" error | Other notebooks consuming RAM | Runtime → Manage sessions → Terminate others |
| Download stalls at 99% | Network timeout | Restart runtime, rerun setup cell |
| "Model not found" error | Hugging Face access issue | Check you're online; try again in 1-2 minutes |
| Slow inference (>2 min/image) | CPU-only mode (no GPU allocated) | Runtime → Change runtime type → T4 GPU (if available) |

**💡 Pro Tip:** If you're repeatedly restarting the runtime during experimentation, the model download is cached. You only pay the download cost once per Colab session.

## 2. Golden Path: First Assisted Read

We'll analyse a sample chest X-ray while explicitly threading in the patient context you curated earlier.

In [ ]:
sample_image_url = "https://staticnew-prod.topdoctors.co.uk/files/Image/large/5ad0cd79-11d0-47b7-840b-123525bbab96.jpg"
clinical_focus = "Monitor pneumonia resolution and cardiac workload"
question = "Describe acute and chronic findings relevant to this patient's recovery plan."
persona = "radiologist"

print("1️⃣ Displaying imaging study:")
load_and_display_image(sample_image_url)

prompt = build_radiology_prompt(
    profile=patient_profile,
    clinical_focus=clinical_focus,
    question=question,
    persona=persona,
)
print("\n2️⃣ Prompt sent to MedGemma:\n")
print(prompt)

report = medgemma_engine.analyze(
    image_path_or_url=sample_image_url,
    prompt=prompt,
    persona=persona,
)

print("\n--- AI PRELIMINARY REPORT ---")
render_ai_report(report)

Take a moment to compare the AI's commentary with your handover note. Where do they agree? Where should a clinician remain sceptical?

### 🔍 Critical Skill: Recognizing AI Hallucinations

Now that you've seen your first AI-generated report, let's practice the most important safety skill: **detecting hallucinations and overconfident statements**.

**What is a Hallucination?**

In AI systems, a **hallucination** occurs when the model generates plausible-sounding content that isn't supported by the input data. In medical imaging, this means reporting findings that aren't actually visible in the image.

**Common Hallucination Patterns in Radiology AI:**

| Pattern | Example | Why It Happens |
|---------|---------|----------------|
| **Fabricated devices** | "ET tube tip at the carina" when no tube is present | Training data heavily featured intubated patients |
| **Phantom pathology** | "Small left pleural effusion" on a normal film | Model associates clinical context with findings |
| **Precise measurements** | "Cardiac silhouette measures 14.2cm" | Model generates numbers without actual measurement |
| **Definitive diagnoses** | "Pneumonia confirmed" vs "Opacity concerning for pneumonia" | Overconfident language from training data |
| **Temporal assumptions** | "New infiltrate since prior study" when no prior was provided | Model fabricates comparison context |

**🧪 Quick Exercise: Audit the Report Above**

Review the AI-generated report from Section 2. For each statement, classify it as:

- ✅ **Descriptive observation** (safe): "The cardiac silhouette appears enlarged"
- ⚠️ **Interpretive finding** (verify): "Findings consistent with pulmonary edema"  
- 🚨 **Definitive diagnosis** (escalate): "Patient has congestive heart failure"
- ❓ **Unprovable claim** (hallucination risk): "ETT tip is 3cm above the carina" (if no ETT visible)

**Which statements would you flag for radiologist verification?**

## 3. Interactive Lab: Prompt Crafting & Personas

You now control the imaging consult. Experiment with different focuses, questions, and personas while keeping track of what helps—or confuses—the AI.

---

### 🎓 What You've Learned So Far

**Section 1-2 Recap:**
- ✅ Initialized a medical vision-language model in Colab
- ✅ Ran your first AI-assisted imaging analysis
- ✅ Learned to recognize hallucinations and verification requirements

**Next:** Now you'll take control. In Section 3, you'll craft your own prompts, experiment with different personas, and discover what makes AI output clinically useful vs unreliable.

**Estimated time for Section 3:** 30-40 minutes

---

### Challenge 3.1: Structured Prompt Engineering

**Clinical Scenario:**

Jordan's care team administered 3 liters of IV fluids during sepsis resuscitation. It's now Day 3, and the team is concerned about **fluid overload**. They need the AI to focus specifically on signs of pulmonary vascular congestion and pleural effusions.

**Your Task:**

Craft a new prompt that:
1. Directs the AI's attention to **fluid status** (not general findings)
2. Asks for **actionable observations** that inform diuresis decisions
3. Maintains clinical context from Jordan's handover note

**📋 Good Medical Prompt Checklist:**

Before you write your prompt, review these principles:

- ✅ **Specific clinical question** (not "describe everything")
  - Good: "Assess for pulmonary edema and pleural effusions"
  - Poor: "Look at the chest X-ray"

- ✅ **Relevant patient context** (from the handover note)
  - Include: Recent fluid balance, current symptoms, treatment course
  - Avoid: Irrelevant biographical details

- ✅ **Requests structured output**
  - Ask for: Findings → Impression → Recommendations
  - Avoid: Open-ended "tell me what you see"

- ✅ **Appropriate scope**
  - Good: "Describe findings concerning for volume overload"
  - Poor: "Should we give furosemide?" (asks AI to prescribe)

**💡 Example Comparison:**

| Aspect | ❌ Weak Prompt | ✅ Strong Prompt |
|--------|--------------|-----------------|
| **Focus** | "Analyze this chest X-ray" | "Assess for pulmonary vascular congestion in a patient with +3L fluid balance" |
| **Context** | None provided | "Post-sepsis resuscitation, currently euvolemic per exam" |
| **Output** | Unstructured | "Provide findings, impression, and suggest follow-up imaging if needed" |
| **Scope** | "What should I do?" | "Describe radiographic findings that inform diuresis decisions" |

**🎯 Success Criteria:**

Your prompt should produce a report that:
- ✅ Mentions specific findings related to fluid status (vascular markings, effusions, Kerley B lines)
- ✅ Connects observations back to the clinical question
- ✅ Uses appropriate hedging language ("suggestive of", "may represent") for ambiguous findings
- ✅ Flags uncertainties rather than guessing
- ❌ Does NOT fabricate devices or measurements
- ❌ Does NOT make treatment decisions

**Now update the variables below and run the analysis:**

In [ ]:
# TODO: Update these variables with your fluid-focused prompt
experiment_focus = "Update this to focus on fluid overload assessment"
experiment_question = "Update this to ask for specific findings related to volume status"
experiment_persona = "radiologist"

# Build and display the prompt
prompt = build_radiology_prompt(
    profile=patient_profile,
    clinical_focus=experiment_focus,
    question=experiment_question,
    persona=experiment_persona,
)
print("=== YOUR PROMPT ===")
print(prompt)
print("\n")

# TODO: Remove the NotImplementedError and uncomment the code below when ready
raise NotImplementedError("Challenge 3.1: Update the variables above, then remove this line and run the analysis.")

# Uncomment these lines after updating the prompt variables:
# experiment_report = medgemma_engine.analyze(
#     image_path_or_url=sample_image_url,
#     prompt=prompt,
#     persona=experiment_persona,
# )
#
# print("=== AI REPORT ===")
# render_ai_report(experiment_report)
#
# print("\n🧪 Self-Assessment: Review the success criteria above.")
# print("Did your prompt produce findings focused on fluid status?")
# print("Did the AI use appropriate hedging language?")
# print("Any hallucinations or overconfident statements to flag?")

### 📝 Prompt Lab Log
Record your trials here. What wording produced the clearest guidance?

In [ ]:
prompt_lab_notes = '''
# Prompt Experimentation Log

**Instructions:** After each experiment (Challenges 3.1-3.3), record what you learned.

| Experiment | Focus | Persona | Useful Takeaways | Cautions |
|------------|-------|---------|------------------|----------|
| **Example** | Assess cardiac silhouette size and pulmonary vasculature | Cardiologist | AI flagged possible cardiomegaly with CTR reference; mentioned pulmonary vascular redistribution | AI estimated CTR without actual measurement tools—would need manual verification before reporting |
| **Challenge 3.1** | <Your fluid overload focus> | <Your persona> | <What worked well?> | <What to verify or be skeptical about?> |
| **Challenge 3.2** | <Your cardiology focus> | <Your persona> | <What changed with the new persona?> | <Any hallucinations or overconfidence?> |
| **Challenge 3.3** | <Your custom image focus> | <Your persona> | <How did different image quality affect output?> | <Any misidentification of modality or view?> |

---

## Key Insights from Your Experiments

**What makes a prompt effective?**
- (Record your observations here after completing the challenges)

**Which persona produced the most clinically useful output for Jordan's case?**
- (Your answer here)

**What findings did you consistently need to verify across all AI reports?**
- (Common hallucination patterns you noticed)

**If you were deploying this AI in a real radiology department, what safeguards would you require?**
- (Your recommendations based on what you learned)

---

💡 **Pro Tip:** Save this log! These insights transfer to other clinical AI tools you'll encounter in practice.
'''
print(prompt_lab_notes)

### Challenge 3.2: Persona Shifting & Priority Framing

**The Power of Persona:**

When you assign a "persona" to the AI, you're essentially asking it to emphasize the priorities and vocabulary of that specialty. Different specialists look at the same image through different clinical lenses.

**How Personas Affect AI Output:**

| Persona | Primary Focus | Typical Language | Common Priorities |
|---------|--------------|------------------|-------------------|
| **Radiologist** | Anatomical findings, differential diagnoses | "Opacity," "consolidation," "silhouette sign" | Comprehensive description, DDx |
| **Cardiologist** | Cardiac silhouette, pulmonary vasculature | "Cardiomegaly," "vascular redistribution" | Heart size, fluid status, device position |
| **Intensivist** | Life-threatening findings, device placement | "Emergent," "critical," "requires immediate" | Pneumothorax, line placement, ARDS |
| **Pulmonologist** | Parenchymal disease, airways | "Infiltrate," "bronchiectasis," "hyperinflation" | Infection, COPD, ILD patterns |

**Clinical Scenario:**

The cardiology team is now rounding on Jordan. They're less concerned with pneumonia resolution (that's improving) and more focused on:
- **Cardiac silhouette size** (concerned about cardiomegaly)
- **Pulmonary vascular markings** (fluid status)
- **Any signs of heart failure** that might complicate sepsis recovery

**Your Task:**

1. Update the persona to `"cardiologist"`
2. Craft a focus statement that prioritizes cardiac concerns
3. Ask a question that a cardiologist would care about (not a general radiologist)

**🎯 Success Criteria:**

The AI report should:
- ✅ Emphasize cardiac findings over pulmonary details
- ✅ Use cardiology-specific terminology (CTR, vascular redistribution, etc.)
- ✅ Comment on findings relevant to heart failure assessment
- ✅ De-emphasize or briefly mention pneumonia (that's pulmonology's domain)

**💡 Pro Tip:**

The best clinical prompts match the AI persona to the actual consult question. If cardiology is consulting for "rule out cardiomegaly," use the cardiologist persona and frame the question accordingly.

In [ ]:
# TODO: Update these variables with a cardiology-focused prompt
persona_focus = "Update with cardiac-specific concerns (cardiac size, heart failure signs)"
persona_question = "Update with a question a cardiologist would ask"
persona_label = "cardiologist"

# Build and display the prompt
prompt = build_radiology_prompt(
    profile=patient_profile,
    clinical_focus=persona_focus,
    question=persona_question,
    persona=persona_label,
)
print("=== CARDIOLOGY-FOCUSED PROMPT ===")
print(prompt)
print("\n")

# TODO: Remove the NotImplementedError and uncomment the code below when ready
raise NotImplementedError("Challenge 3.2: Update the variables above, then remove this line and run the analysis.")

# Uncomment these lines after updating the prompt variables:
# persona_report = medgemma_engine.analyze(
#     image_path_or_url=sample_image_url,
#     prompt=prompt,
#     persona=persona_label,
# )
#
# print("=== CARDIOLOGY-FOCUSED REPORT ===")
# render_ai_report(persona_report)
#
# print("\n🧪 Self-Assessment:")
# print("Compare this report to the radiologist report from Section 2.")
# print("- Which findings did the cardiologist persona emphasize?")
# print("- Which findings were de-emphasized or omitted?")
# print("- Did the language and priorities shift appropriately?")

### Challenge 3.3: Bring Your Own Image & Test Edge Cases

**Time to Stress-Test the AI:**

Now that you understand prompt engineering and persona selection, let's see how the AI handles different types of images. This challenge teaches you to recognize when AI performs well vs. when you should be skeptical.

**Your Task:**

1. **Find a public medical image** (chest X-ray, CT, or other modality) that's relevant to Jordan's case or a similar clinical scenario
2. **Adjust your prompt** to match the image and clinical context
3. **Run the full workflow** and critically evaluate the output

**📸 Where to Find Medical Images:**

- **Radiopaedia:** https://radiopaedia.org/ (free cases with teaching files)
- **Open-i NIH:** https://openi.nlm.nih.gov/ (biomedical image search)
- **Wikipedia Commons:** Medical imaging category (public domain)

**⚠️ Image Selection Tips:**

Different image types will challenge the AI in different ways:

| Image Type | What It Tests | Expected AI Performance |
|------------|--------------|------------------------|
| **High-quality PA chest X-ray** | Baseline capability | Should perform well |
| **Portable/AP chest X-ray** | Lower quality tolerance | May struggle with subtle findings |
| **Lateral chest X-ray** | View interpretation | May describe as if it's PA view (hallucination!) |
| **CT scan** | Modality recognition | May confuse findings or use wrong terminology |
| **Pediatric image** | Age-specific findings | May apply adult reference ranges |
| **Image with poor exposure** | Artifacts & quality issues | Higher hallucination risk |

**🎯 Learning Objectives:**

By running this challenge, you'll discover:
- ✅ Which image qualities produce reliable AI output
- ✅ How the AI handles modality and view variations
- ✅ When you should have lower confidence in AI findings
- ✅ The importance of matching clinical context to image type

**💡 Suggested Experiment:**

Try two contrasting images:
1. A **high-quality teaching file** (optimal conditions for AI)
2. A **portable/bedside film** (realistic ICU conditions)

Compare the AI's confidence, specificity, and accuracy between the two.

In [ ]:
# TODO: Update these variables with your chosen image and customized prompt
your_image_url = "https://example.com/your-chosen-image.jpg"  # Replace with actual URL
your_focus = "Tailor this focus to match the clinical scenario for your chosen image"
your_question = "What specific clinical question does this image need to answer?"
your_persona = "radiologist"  # Or cardiologist, pulmonologist, etc.

# Display the image first
print("=== YOUR CHOSEN IMAGE ===")
load_and_display_image(your_image_url)

# Build and display the prompt
prompt = build_radiology_prompt(
    profile=patient_profile,
    clinical_focus=your_focus,
    question=your_question,
    persona=your_persona,
)
print("\n=== YOUR PROMPT ===")
print(prompt)
print("\n")

# TODO: Remove the NotImplementedError and uncomment the code below when ready
raise NotImplementedError("Challenge 3.3: Update the variables above (especially your_image_url), then remove this line.")

# Uncomment these lines after updating all variables:
# your_report = medgemma_engine.analyze(
#     image_path_or_url=your_image_url,
#     prompt=prompt,
#     persona=your_persona,
# )
#
# print("=== AI REPORT FOR YOUR IMAGE ===")
# render_ai_report(your_report)
#
# print("\n🧪 Critical Evaluation Checklist:")
# print("[ ] Did the AI correctly identify the image modality (X-ray vs CT vs other)?")
# print("[ ] Did the AI correctly identify the view (PA vs AP vs lateral)?")
# print("[ ] Are there any findings the AI reported that you can't see in the image?")
# print("[ ] Did image quality affect the AI's confidence or specificity?")
# print("[ ] Would you trust this report enough to escalate to the attending, or does it need revision?")

## 4. Advanced Mission: Analyse Your Own Data

If a remote URL fails—or you want to test a de-identified case from your institution—upload it directly.

---

### 🎓 Section 3 Checkpoint

**What You've Practiced:**
- ✅ Prompt engineering for specific clinical questions
- ✅ Persona shifting to match consult scenarios
- ✅ Evaluating AI performance across different image types

**Key Insight:** The quality of AI output depends heavily on how you frame the question and what context you provide.

**Next:** Section 4 shows you how to work with locally uploaded images (useful for de-identified institutional cases).

**Estimated time for Section 4:** 10-15 minutes

---

In [ ]:
from google.colab import files

print("Please upload an image file (e.g., .jpg, .png).")
uploaded = files.upload()

if uploaded:
    local_image_path = next(iter(uploaded))
    print(f"Displaying uploaded image: {local_image_path}")
    load_and_display_image(local_image_path)

    local_focus = "Describe why you captured this image"
    local_question = "What actionable insights do you need before rounds?"

    prompt = build_radiology_prompt(
        profile=patient_profile,
        clinical_focus=local_focus,
        question=local_question,
        persona="radiologist",
    )

    local_report = medgemma_engine.analyze(
        image_path_or_url=local_image_path,
        prompt=prompt,
        persona="radiologist",
    )

    print("--- AI REPORT (LOCAL IMAGE) ---")
    render_ai_report(local_report)
else:
    raise FileNotFoundError("Upload cancelled. Please rerun the cell and choose an image.")


## 5. Extension: Benchmarking Against the Competition

MedVision's R&D team wants a head-to-head comparison between MedGemma (production) and LLaVA (experimental). Keep your evaluation clinically grounded.

---

### 🎓 Section 4 Checkpoint

**What You've Practiced:**
- ✅ Working with locally uploaded images (not just URLs)
- ✅ Adapting prompts to custom clinical scenarios

**Next:** Section 5 is the capstone—you'll compare two competing AI models using a clinical decision framework and make a deployment recommendation to the MedVision leadership team.

This is where you synthesize everything you've learned about prompt engineering, hallucination detection, and clinical evaluation.

**Estimated time for Section 5:** 20-30 minutes

---

### Benchmark Protocol
1. Form a hypothesis: where do you expect MedGemma to outperform LLaVA?
2. Keep the patient context, image, and prompt identical across engines.
3. Evaluate structure, accuracy, and bedside usefulness.
4. Capture findings in the comparison log.

### How to Evaluate Medical AI: A Clinical Decision Framework

When comparing AI models for clinical deployment, **technical metrics alone aren't enough**. A model with impressive benchmark scores might still be unsafe if it hallucinates critical findings or uses overconfident language.

Use this clinical evaluation rubric to assess both models:

#### **Clinical Evaluation Rubric**

| Criterion | Weight | What to Assess | Example |
|-----------|--------|----------------|---------|
| **🔴 Accuracy** | Critical | Does the model correctly identify findings that ARE present? | Model identifies consolidation visible in RLL |
| **🔴 Safety** | Critical | Does the model avoid hallucinating findings that AREN'T present? | Model doesn't report ETT when none is visible |
| **🟡 Specificity** | Important | Does the model use precise anatomical/radiological terminology? | "RLL consolidation" vs "white spot in lung" |
| **🟡 Actionability** | Important | Do findings inform clinical decisions? | "Small effusion, clinical correlation advised" vs "Some fluid maybe" |
| **🟡 Appropriate Uncertainty** | Important | Does the model say "unclear" when image quality limits assessment? | "Poor inspiration limits evaluation of bases" |
| **🟡 Structured Output** | Important | Is the report organized (Findings → Impression → Recommendations)? | Clear sections vs stream of consciousness |
| **🟢 Formatting** | Nice-to-have | Is the output scannable and easy to read? | Bullet points, clear paragraphs |

**Priority Order:**
1. **Safety first:** A model that hallucinates less is better than one with prettier formatting
2. **Clinical utility:** Reports must be actionable for the care team
3. **Usability:** Structure and formatting enhance (but don't replace) clinical value

---

#### **How to Make Your Deployment Recommendation**

After running both models, ask yourself:

**For Safety & Accuracy (Most Important):**
- Which model made fewer safety-critical errors (missed findings, hallucinations)?
- Which model was more conservative with uncertainty (appropriately hedged)?
- If you had to escalate ONE report to the attending, which would it be?

**For Clinical Utility:**
- Which model provided findings that directly address the clinical question?
- Which model connected findings to patient context more effectively?
- Which model's output would save the radiologist the most time?

**For Trust & Transparency:**
- Which model's outputs were more consistent across different prompts?
- Which model was easier to "steer" with persona and focus adjustments?
- If an error occurred, which model's mistake would be easier to catch?

**Your Final Recommendation Should State:**
1. **Which model to deploy** (MedGemma or LLaVA)
2. **Why** (specific strengths observed)
3. **What safeguards are required** (human review protocol, specific findings to always verify)
4. **What limitations remain** (edge cases where neither model is reliable)

---

**Now let's run the benchmark with these criteria in mind.**

In [ ]:
from medvision_toolkit import initialize_llava_engine

# Step 1: Initialize the experimental LLaVA engine
print("--- Initialising LLaVA Experimental Engine ---")
print("⏱️ This may take 3-5 minutes on first run (downloading model weights)...\n")

# TODO: Remove the NotImplementedError below when ready to run the benchmark
raise NotImplementedError("Challenge 5: Remove this line to initialize LLaVA and run the benchmark comparison.")

# Uncomment the code below after removing the NotImplementedError:
#
# llava_engine = initialize_llava_engine()
# print("✅ LLaVA engine ready.\n")
#
# # Step 2: Define a test case (same image and prompt for both models)
# test_image_url = "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"
# test_focus = "Assess for interval changes, urgent safety findings, and signs of infection"
# test_question = "Provide a structured report with findings, impression, and follow-up recommendations."
# test_persona = "radiologist"
#
# # Build the standardized prompt
# prompt = build_radiology_prompt(
#     profile=patient_profile,
#     clinical_focus=test_focus,
#     question=test_question,
#     persona=test_persona,
# )
#
# print("=== BENCHMARK TEST CASE ===")
# print(f"Image: {test_image_url}")
# load_and_display_image(test_image_url)
# print("\n=== STANDARDIZED PROMPT (Same for Both Models) ===")
# print(prompt)
# print("\n" + "="*80)
#
# # Step 3: Run MedGemma analysis
# print("\n🔬 Running MedGemma analysis...")
# print("(This may take 30-60 seconds)\n")
# medgemma_report = medgemma_engine.analyze(
#     image_path_or_url=test_image_url,
#     prompt=prompt,
#     persona=test_persona,
# )
#
# # Step 4: Run LLaVA analysis
# print("\n🔬 Running LLaVA analysis...")
# print("(This may take 30-60 seconds)\n")
# llava_report = llava_engine.analyze(
#     image_path_or_url=test_image_url,
#     prompt=prompt,
# )
#
# # Step 5: Display both reports side-by-side
# print("\n" + "="*80)
# print("📊 BENCHMARK RESULTS")
# print("="*80)
#
# print("\n--- [PRODUCTION] MedGemma Report ---")
# print("-" * 80)
# render_ai_report(medgemma_report)
#
# print("\n\n--- [EXPERIMENTAL] LLaVA Report ---")
# print("-" * 80)
# render_ai_report(llava_report)
#
# print("\n" + "="*80)
# print("\n🧪 Now complete the benchmark comparison log above.")
# print("Use the clinical evaluation rubric to assess both models systematically.")
# print("Focus on: Safety > Accuracy > Actionability > Structure")

### 📝 Benchmark Comparison Log
Summarise the relative strengths, limitations, and deployment decision.

In [ ]:
benchmark_notes = '''
# Model Comparison & Deployment Decision

**Instructions:** After running both MedGemma and LLaVA on the same test case, complete this evaluation.

---

## Side-by-Side Comparison

| Criterion | MedGemma | LLaVA | Winner |
|-----------|----------|-------|--------|
| **🔴 Accuracy** | <Correctly identified findings?> | <Correctly identified findings?> | <Which was more accurate?> |
| **🔴 Safety (No Hallucinations)** | <Any fabricated findings?> | <Any fabricated findings?> | <Which hallucinated less?> |
| **🟡 Specificity** | <Precise terminology?> | <Precise terminology?> | <Which was more specific?> |
| **🟡 Actionability** | <Clinically useful?> | <Clinically useful?> | <Which was more actionable?> |
| **🟡 Appropriate Uncertainty** | <Hedged appropriately?> | <Hedged appropriately?> | <Which handled uncertainty better?> |
| **🟡 Structure** | <Well-organized report?> | <Well-organized report?> | <Which had better structure?> |

---

## Detailed Observations

### MedGemma Strengths:
- (What did MedGemma do particularly well?)

### MedGemma Limitations:
- (What were MedGemma's weaknesses or errors?)

### LLaVA Strengths:
- (What did LLaVA do particularly well?)

### LLaVA Limitations:
- (What were LLaVA's weaknesses or errors?)

---

## Deployment Recommendation

**Recommended Model for Clinical Deployment:** <MedGemma / LLaVA / Neither>

**Rationale:**
- (Why did you choose this model? Focus on safety and clinical utility)

**Required Safeguards:**
1. (What human oversight is required?)
2. (Which findings must always be verified by a radiologist?)
3. (What patient populations or image types should be excluded?)

**Remaining Concerns:**
- (What would need to improve before you'd trust this AI in practice?)
- (Are there edge cases where neither model is reliable?)

---

## Key Takeaway

If you had to summarize your learning from this benchmark in one sentence for the MedVision leadership team:

**"<Your one-sentence recommendation here>"**

---

**Example Completed Entry:**

Model: MedGemma
Strengths: Conservative language ("may represent"), good structure, identified major findings
Limitations: Slightly verbose, estimated cardiac measurements without tools
Winner for safety-critical deployment: MedGemma (fewer hallucinations)

'''
print(benchmark_notes)

### Solution (Click to Expand)

Run this cell only after attempting the benchmark yourself.

In [ ]:

print("--- Initialising Llava Experimental Engine ---")
llava_engine = initialize_llava_engine()

test_image_url = "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"
test_focus = "Assess interval changes and urgent safety findings"
test_question = "Provide a structured report with impression and follow-up recommendations."

prompt = build_radiology_prompt(
    profile=patient_profile,
    clinical_focus=test_focus,
    question=test_question,
    persona="radiologist",
)

print("\nRunning MedGemma analysis...")
medgemma_report = medgemma_engine.analyze(
    image_path_or_url=test_image_url,
    prompt=prompt,
    persona="radiologist",
)

print("\nRunning LLaVA analysis...")
llava_report = llava_engine.analyze(
    image_path_or_url=test_image_url,
    prompt=prompt,
)

print("\n" + "=" * 80)
print("--- [PRODUCTION] MedGemma Report ---")
print("=" * 80)
print(medgemma_report)
print("\n" + "=" * 80)
print("--- [EXPERIMENTAL] LLaVA Report ---")
print("=" * 80)
print(llava_report)


## 6. Clinical Debrief: From Lab to Practice

### What You've Accomplished

Over the past two notebooks, you've built a complete clinical AI workflow:

**Notebook 01: Python Fundamentals**
- ✅ Structured patient data using variables, lists, and dictionaries
- ✅ Automated triage logic with loops and conditionals
- ✅ Generated handover notes from typed data structures

**Notebook 02: AI-Assisted Radiology**
- ✅ Understood how vision-language models work (and their limitations)
- ✅ Initialized and operated a medical VLM in a resource-constrained environment
- ✅ Crafted prompts that ground AI analysis in clinical context
- ✅ Recognized hallucinations and applied safety verification protocols
- ✅ Evaluated competing models using clinical decision criteria
- ✅ Made an evidence-based deployment recommendation

**This is no small feat.** You've gone from zero Python experience to operating cutting-edge AI tools with clinical discipline.

---

### Critical Insights to Take Forward

#### **On AI Capabilities:**

**What AI Does Well:**
- Rapid pattern recognition across large imaging datasets
- Drafting preliminary reports that save radiologist time
- Flagging potential findings for human review (triage)
- Maintaining consistency in terminology and structure

**What AI Struggles With:**
- Rare or unusual pathologies not well-represented in training data
- Precise measurements (angles, volumes, dimensions) without measurement tools
- Integrating clinical context beyond what's explicitly stated in the prompt
- Recognizing when image quality limits assessment

**The Bottom Line:** AI is a powerful assistant, not a replacement. It excels at the 80% of routine cases but needs human expertise for the nuanced 20%.

---

#### **On Prompt Engineering:**

**What You Learned:**

1. **Specificity Matters:**
   - "Assess for pulmonary edema" > "Look at the chest X-ray"
   - The more specific your clinical question, the more focused the AI response

2. **Context Transforms Output:**
   - Providing patient history changes what the AI considers clinically relevant
   - The same image analyzed with different context produces different findings

3. **Persona Framing Works:**
   - Assigning a specialist role (cardiologist vs radiologist) shifts priorities
   - Match the persona to the actual consult question for best results

4. **Structure Improves Usability:**
   - Asking for "Findings → Impression → Recommendations" produces actionable reports
   - Open-ended prompts lead to rambling, less useful outputs

**Transferable Skill:** These prompt engineering principles apply to all clinical AI tools, not just radiology.

---

#### **On Safety & Verification:**

**Hallucinations Are Common:**
- All VLMs occasionally report findings that aren't present
- Devices, measurements, and temporal comparisons are high-risk areas
- The more specific/confident the AI sounds, the more skeptical you should be

**Trust, But Verify:**
- Every AI finding requires correlation with source images
- Critical findings (pneumothorax, free air, PE) require immediate human review
- Document that AI was used as a decision support tool, not a diagnostic authority

**Scope of Practice:**
- As students/junior doctors, you're learning to *consume* AI outputs critically
- Interpreting studies and making diagnoses remains the radiologist's domain
- Know when to escalate vs when to trust preliminary AI reads

---

### Reflection Exercise

Before closing this notebook, take 5 minutes to answer these questions for yourself. You can write your responses in a new markdown cell below, or in your personal notes.

**1. Clinical Judgment:**

*"Under what circumstances would you trust this AI enough to flag a case as 'likely normal' and deprioritize it in the reading queue? What safeguards would you require?"*

**2. Workflow Integration:**

*"Where in the care pathway would AI-assisted reads add the most value at your institution? (Screening? Overnight triage? Teaching files? Quality assurance?)"*

**3. Ethical Responsibility:**

*"If an AI-assisted read missed a finding that a radiologist would have caught, who is responsible? How does this affect your comfort level using these tools?"*

**4. Personal Learning:**

*"What surprised you most about working with medical AI? What remains concerning or unclear?"*

---

### Next Steps in Your AI-Augmented Practice

#### **In This Course (Optional Extensions):**

- **Stress-test the models:** Repeat the benchmark with your own de-identified cases or challenging public datasets
- **Build a prompt library:** Create reusable prompt templates for common clinical scenarios (PE protocol, pneumonia follow-up, post-op chest)
- **Share your findings:** Present your benchmark results to your learning group or clinical supervisor

#### **In Your Clinical Training:**

- **Seek out AI tools:** Ask your radiology, pathology, and cardiology departments what AI systems they're using
- **Observe integration:** Watch how attendings incorporate AI into their workflow (Do they review every AI flag? Ignore certain outputs?)
- **Advocate for transparency:** If AI influences a decision about your patient, make sure it's documented in the chart

#### **Stay Current:**

- **Follow the literature:** Medical AI is evolving rapidly; what's true today may change in 6 months
- **Attend AI grand rounds:** Many academic centers now host regular sessions on clinical AI
- **Engage with policy:** Regulatory frameworks for medical AI are being written *now*; your generation will inherit (or revise) them

---

### Final Thought: Stay Curious, Stay Critical

You now have the foundation to be a **thoughtful consumer and collaborator** with medical AI. You understand:

- How these systems work (pattern recognition, not reasoning)
- Where they excel (routine cases, rapid triage)
- Where they fail (hallucinations, rare pathology)
- How to evaluate them (safety > accuracy > utility)

**The technology will evolve.** Models will get better, faster, and more specialized. But your responsibility to **verify, question, and prioritize patient safety** will not change.

AI is a tool. Like a stethoscope, it's only as good as the clinician wielding it.

**You've earned the right to wield it well. Congratulations on completing Project MedVision.**

---

### Optional: Document Your Work

If you'd like to save your experiments for future reference:

1. **Download this notebook:** File → Download → Download .ipynb
2. **Save your logs:** Copy your prompt lab notes and benchmark notes to a personal document
3. **Capture key examples:** Screenshot particularly good or bad AI outputs for teaching cases

Your future self (or your med school study group) will thank you.